Instalação das packages e importação das mesmas

In [1]:
%pip install transformers datasets torch optuna sentencepiece wandb
import torch
import optuna
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, Trainer, TrainingArguments, pipeline, EarlyStoppingCallback
from datasets import load_dataset
import wandb
from transformers.integrations import WandbCallback

Note: you may need to restart the kernel to use updated packages.


Login no weights & biases para fazer plot do treino

In [2]:
wandb.login(key="fd025546a653d692020ef72e4c1779479088b079") # maneira mais segura de passar uma key

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/azureuser/.netrc


True

Load do dataset "_multi news_" da bilbioteca _datasets_ que contem notícias e o respetivo sumário
<br>O dataset vem com 3 splits, _train, validation e test_
<br>Para o treino e selecionou-se só parte do dataset devido a este ser muito grande e necessitar de demasiada memória no treino do modelo


In [3]:
dataset = load_dataset("multi_news") #load ao dataset

trainPercentage = int(1 * len(dataset["train"])) #arranjar x% do tamanho do dataset

trainSplit = dataset["train"].select(range(trainPercentage)) #selecionar só x% do dataset de treino
validationSplit = dataset["validation"] #seleccionar o dataset de validação

Found cached dataset multi_news (/home/azureuser/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
t5 = "t5-base"

model = T5ForConditionalGeneration.from_pretrained(t5) # carrega o modelo T5 
tokenizer = T5Tokenizer.from_pretrained(t5) # carrega o tokenizer T5 com o vocabulário pré-treinado do t5-base

Função de tokenização usando o modelo  _T5Tokenizer_ e ficheiro de vocabulário _t5-base_
<br> Retorna _input ids, attention mask e labels_ que são usados no treino do modelo
<br> Feita a tokenização do dataset de treino e de validação

In [4]:
def preprocess(data):
    tokenizedInput = tokenizer(data['document'], padding=True, truncation=True, return_tensors="pt", max_length=512) # faz tokenização das notícias, o que será a "entrada" na nossa aplicação, pt pq estamos a usar pytorch

    tokenizedTarget = tokenizer(data['summary'], padding=True, truncation=True, return_tensors="pt", max_length=150) # faz tokenização dos sumários, ou "saída", alterar o pt p tf se formos usar tensorflow

    return {'input_ids': tokenizedInput['input_ids'], 'attention_mask': tokenizedInput['attention_mask'], 
            'labels': tokenizedTarget['input_ids']} # t5 necessita de input ids, attention mask e labels para treinar

tokenizedTrain = trainSplit.map(preprocess, batched=True, batch_size=16) # aqui o map vai aplicar a tokenização a todos os elementos do dataset
tokenizedValidation = validationSplit.map(preprocess, batched=True, batch_size=16) # aqui o map vai aplicar a tokenização a todos os elementos do dataset

Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72/cache-96184cf5118120ec.arrow
Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72/cache-b1a40954ff931d0c.arrow


Aqui é definido os parametros de treino do modelo, é criado o modelo e é treinado o modelo com variação de hiperparametros

In [6]:
def model_training(trial):
    # Teste hyierparametros, 3 valores diferentes para learning rate, weight decay e epochs, usados com combinações aleatórias 
    learningRate = trial.suggest_categorical(
        "learning_rate", [1e-5, 2e-5 ,5e-5, 1e-4])
    weightDecay = trial.suggest_categorical("weight_decay", [0.0, 0.01, 0.005, 0.001])
    epochs = trial.suggest_categorical("num_epochs", [1,2,3])
       
    args = TrainingArguments(
        # tipo de avaliação do modelo, estamos a utilizar avaliação por epoch
        evaluation_strategy="epoch",
        # numero de vezes que o dataset vai ser corrido no treino
        num_train_epochs=epochs,
        # batch size refere-se à quantidade de samples que são processadas de uma vez pelo gpu neste caso
        per_device_train_batch_size=4,
        # batch size refere-se à quantidade de samples que são processadas de uma vez pelo gpu neste caso
        per_device_eval_batch_size=4,
        # usado p reduzir batches em mini batches quando fica muito pesado em termos de memória
        gradient_accumulation_steps=4,
        fp16=True,  # mixed precision, torna o modelo mais eficiente ao utilizar valores float de 16 e 32 bits, o que pode aumentar velocidade e torna mais eficiente em memória, só funciona se for utilizado CUDA
        save_strategy="epoch",
        save_total_limit=10,  # nm de modelos que grava
        # pensar no declive, quanto maior pode saltar dados e n converge, quanto menor pode demorar mt a convergir e pode não atingir min globais
        learning_rate=learningRate,
        weight_decay=weightDecay,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        output_dir="./results",
        logging_dir="./logs",
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenizedTrain,
        eval_dataset=tokenizedValidation,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2),WandbCallback], #define quantos epochs precisa sem melhoria para parar de treinar 
    )

    trainer.train()
    metrics = trainer.evaluate()
    return metrics["eval_loss"]

In [7]:
wandb.init()
aval = optuna.create_study(direction="minimize") #otimizar hiperparametros para minimizar loss do modelo
aval.optimize(model_training, n_trials=10) #quantas combinações de hiperparametros vai fazer, se for 10 ele vai fazer 10 combinações possíveis

wandb: Currently logged in as: 1190593 (aitiv8). Use `wandb login --relogin` to force relogin


[I 2023-04-14 11:43:14,712] A new study created in memory with name: no-name-394cef1c-dbe4-4d0d-8b96-2bb5b3bd2834
You are adding a <class 'transformers.integrations.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
AzureMLCallback
TensorBoardCallback
WandbCallback
EarlyStoppingCallback
/anaconda/envs/jupyter_env/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,2.885274
2,No log,2.753945
2,No log,2.730504


Attempted to log scalar metric eval_loss:
2.8852741718292236
Attempted to log scalar metric eval_runtime:
1271.1953
Attempted to log scalar metric eval_samples_per_second:
4.423
Attempted to log scalar metric eval_steps_per_second:
1.106
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric eval_loss:
2.7539453506469727
Attempted to log scalar metric eval_runtime:
1270.8467
Attempted to log scalar metric eval_samples_per_second:
4.424
Attempted to log scalar metric eval_steps_per_second:
1.106
Attempted to log scalar metric epoch:
2.0
Attempted to log scalar metric eval_loss:
2.730504035949707
Attempted to log scalar metric eval_runtime:
1271.8555
Attempted to log scalar metric eval_samples_per_second:
4.42
Attempted to log scalar metric eval_steps_per_second:
1.105
Attempted to log scalar metric epoch:
2.99
Attempted to log scalar metric train_runtime:
8344.6234
Attempted to log scalar metric train_samples_per_second:
0.808
Attempted to log scalar metric train_steps

[I 2023-04-14 14:23:35,071] Trial 0 finished with value: 2.730504035949707 and parameters: {'learning_rate': 1e-05, 'weight_decay': 0.005, 'num_epochs': 3}. Best is trial 0 with value: 2.730504035949707.
You are adding a <class 'transformers.integrations.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
AzureMLCallback
TensorBoardCallback
WandbCallback
EarlyStoppingCallback
/anaconda/envs/jupyter_env/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Attempted to log scalar metric eval_loss:
2.5944979190826416
Attempted to log scalar metric eval_runtime:
1271.9593
Attempted to log scalar metric eval_samples_per_second:
4.42
Attempted to log scalar metric eval_steps_per_second:
1.105
Attempted to log scalar metric epoch:
1.99
Attempted to log scalar metric eval_loss:
2.8151304721832275
Attempted to log scalar metric eval_runtime:
1273.9653
Attempted to log scalar metric eval_samples_per_second:
4.413
Attempted to log scalar metric eval_steps_per_second:
1.104
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric train_runtime:
2791.5506
Attempted to log scalar metric train_samples_per_second:
0.805
Attempted to log scalar metric train_steps_per_second:
0.05
Attempted to log scalar metric total_flos:
1364065674854400.0
Attempted to log scalar metric train_loss:
1.3928147452218191
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric eval_loss:
2.8151304721832275
Attempted to log scalar metric eva

[I 2023-04-15 04:46:11,531] Trial 8 finished with value: 2.8151304721832275 and parameters: {'learning_rate': 1e-05, 'weight_decay': 0.001, 'num_epochs': 1}. Best is trial 1 with value: 2.477335214614868.
You are adding a <class 'transformers.integrations.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
AzureMLCallback
TensorBoardCallback
WandbCallback
EarlyStoppingCallback
/anaconda/envs/jupyter_env/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,2.848568
2,No log,2.666286
2,No log,2.616757


Attempted to log scalar metric eval_loss:
2.8485679626464844
Attempted to log scalar metric eval_runtime:
1273.6975
Attempted to log scalar metric eval_samples_per_second:
4.414
Attempted to log scalar metric eval_steps_per_second:
1.104
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric eval_loss:
2.6662864685058594
Attempted to log scalar metric eval_runtime:
1273.614
Attempted to log scalar metric eval_samples_per_second:
4.414
Attempted to log scalar metric eval_steps_per_second:
1.104
Attempted to log scalar metric epoch:
2.0
Attempted to log scalar metric eval_loss:
2.6167571544647217
Attempted to log scalar metric eval_runtime:
1273.4048
Attempted to log scalar metric eval_samples_per_second:
4.415
Attempted to log scalar metric eval_steps_per_second:
1.104
Attempted to log scalar metric epoch:
2.99
Attempted to log scalar metric train_runtime:
8295.52
Attempted to log scalar metric train_samples_per_second:
0.813
Attempted to log scalar metric train_steps_

Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/anaconda/envs/jupyter_env/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/anaconda/envs/jupyter_env/lib/python3.8/site-packages/wandb/sdk/service/server_sock.py", line 100, in run
    sreq = self._sock_client.read_server_request()
  File "/anaconda/envs/jupyter_env/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 274, in read_server_request
    data = self._read_packet_bytes()
  File "/anaconda/envs/jupyter_env/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 248, in _read_packet_bytes
    rec = self._extract_packet_bytes()
  File "/anaconda/envs/jupyter_env/lib/python3.8/site-packages/wandb/sdk/lib/sock_client.py", line 230, in _extract_packet_bytes
    assert magic == ord("W")
AssertionError
Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/anaconda/envs/jupyter_env/lib/python3.8/threading.py", line 932, in _bootstr

In [8]:
best_params = aval.best_params #melhores hiperparametros de todos os testes feitos
print("Best hyperparameters:", best_params)

Best hyperparameters: {'learning_rate': 0.0001, 'weight_decay': 0.01, 'num_epochs': 3}


In [9]:
def summarize(text):
    summarizer = pipeline("summarization", model="./results/checkpoint-843", tokenizer="t5-base")

    t5_input = "summarize: " + text # o modelo t5 precisa de ter "summarize: " como prefixo p sumarizar

    summary = summarizer(t5_input, min_length=30, max_length=150, num_beams=4, early_stopping=True)
    summarized_text = summary[0]["summary_text"]

    print(summarized_text)
